In [1]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.compat.v1 import ConfigProto
from sklearn.preprocessing import LabelBinarizer
from tensorflow.compat.v1 import InteractiveSession
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [5]:
model = DenseNet121(include_top = False, weights = 'imagenet', input_shape=(128,128,3), pooling = 'max')

model.trainable = False

set_trainable = False
for layer in model.layers:
  if layer.name.startswith("conv5"):
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

x = model.output

bin_classifier = Dense(64, kernel_regularizer = l2(0.05), activation = "relu")(x)
bin_classifier = BatchNormalization(axis = -1)(bin_classifier)
bin_classifier = Dense(1, kernel_regularizer = l2(0.05), activation = "sigmoid")(bin_classifier)

reg_head = Dense(64, kernel_regularizer = l2(0.05), activation = "relu")(x)
reg_head = BatchNormalization(axis = -1)(reg_head)
reg_head = Dense(1, name = "reg_head", kernel_regularizer = l2(0.05))(reg_head)

base_model = Model(model.input, [bin_classifier, reg_head])

base_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 134, 134, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [7]:
images = np.load('../imfdb-dataset/imfdb_images.npy')
gender = np.load('../imfdb-dataset/imfdb_gender_labels.npy')
age = np.load('../imfdb-dataset/imfdb_age_labels.npy')

In [ ]:
images = images.astype("float") / 255.0

In [ ]:
(x_train, x_test, gender_train, gender_test, age_train, age_test) = \
train_test_split(images, gender, age, test_size = 0.2, random_state = 42)

In [ ]:
lb = LabelBinarizer()
gender_train = lb.fit_transform(gender_train)
gender_test = lb.fit_transform(gender_test)

In [ ]:
loss_weights = {'reg_head': 1., 'bin_classifier': 6.}
losses = {'reg_head': 'mse', 'bin_classifier': 'binary_crossentropy'}

In [ ]:
base_model.compile(optimizer = "nadam", loss = losses, loss_weights = loss_weights, metrics = ["acc"])


In [ ]:
chkpt = ModelCheckpoint(filepath = "model_initial.h5", monitor = 'val_bin_classifier_acc',\
                        mode = 'max', save_best_only = True, verbose = 1)
early = EarlyStopping(monitor = "val_acc", mode = "max", patience = 5, verbose = 1)
redonplat = ReduceLROnPlateau(monitor = "val_acc", mode = "max", patience = 3, verbose = 2)

callbacks_list = [chkpt, early, redonplat]

In [ ]:
epochs = 25
batch_size = 16

In [ ]:
base_model.fit(x_train, {'reg_head': age_train, 'bin_classifier': gender_train}, batch_size = batch_size, epochs = epochs, callbacks = callbacks_list, validation_data = (x_test, {'reg_head': age_test, 'bin_classifier': gender_test}))

In [ ]:
np.unique(age_train)